In [31]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, \
    BatchNormalization, LeakyReLU, ReLU, Flatten, Dropout, Dense, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt

In [32]:
# define the standalone discriminator model
def define_discriminator(in_shape=(32, 32, 3)):

    model = Sequential()

    #Conv2D-1
    model.add(Conv2D(filters=64,
                     kernel_size=(3, 3),
                     padding='same',
                     input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))

    #Conv2D-2
    model.add(Conv2D(filters=128,
                     kernel_size=(3,3),
                     strides=(2,2),
                     padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    #Conv2D-3
    model.add(Conv2D(filters=128,
                     kernel_size=(3,3),
                     strides=(2,2),
                     padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    #Conv2D-4
    model.add(Conv2D(filters=256,
                     kernel_size=(3,3),
                     strides=(2,2),
                     padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt,
                  metrics=['accuracy'])
    return model

In [33]:
# visualise the discriminator
d_model = define_discriminator()
d_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
leaky_re_lu_32 (LeakyReLU)   (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 8, 8, 128)         147584    
_________________________________________________________________
leaky_re_lu_34 (LeakyReLU)   (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 4, 4, 256)        

In [34]:
def load_real_samples():

    # load cifar10
    (trainx, trainy), (testx, testy) = cifar10.load_data()
    X = trainx.astype('float32')
    X = (X - 127.5) / 127.5
    return X

In [35]:
# select real samples
def generate_real_samples(dataset:np.ndarray, n_samples):

    # choose random instances
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    x = dataset[ix]
    y = np.ones((n_samples, 1))
    return x, y

In [36]:
def define_generator(latent_dim):
    model = Sequential()

    # foundation for 4x4 image
    n_nodes = 256*4*4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 256)))

    # upsample to 8x8
    model.add(Conv2DTranspose(filters=128,
                              kernel_size=(4, 4),
                              strides=(2, 2),
                              padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    # upsample to 16x16
    model.add(Conv2DTranspose(filters=128,
                              kernel_size=(4, 4),
                              strides=(2, 2),
                              padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    # upsample 32x32
    model.add(Conv2DTranspose(filters=128,
                              kernel_size=(4,4),
                              strides=(2,2),
                              padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    # output layer
    model.add(Conv2D(filters=3,
                     kernel_size=(3,3),
                     activation='tanh',
                     padding='same'))

    return model

In [37]:
g_model = define_generator(latent_dim=100)
g_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 4096)              413696    
_________________________________________________________________
leaky_re_lu_36 (LeakyReLU)   (None, 4096)              0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 8, 8, 128)         524416    
_________________________________________________________________
leaky_re_lu_37 (LeakyReLU)   (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 16, 16, 128)       262272    
_________________________________________________________________
leaky_re_lu_38 (LeakyReLU)   (None, 16, 16, 128)     

In [38]:
# genrate points in the latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)

    return x_input

In [39]:
def generate_fake_samples(g_model:Model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    x = g_model.predict(x_input)
    y = np.zeros((n_samples, 1))

    return x, y

In [40]:
def define_gan(g_model:Model, d_model:Model):
    d_model.trainable = False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)

    return model

In [41]:
latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
gan_model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_12 (Sequential)   (None, 32, 32, 3)         1466115   
_________________________________________________________________
sequential_11 (Sequential)   (None, 1)                 522497    
Total params: 1,988,612
Trainable params: 1,466,115
Non-trainable params: 522,497
_________________________________________________________________


In [ ]:
def save_plot(examples, epoch, n=10):
    # plot images
    for i in range(n*n):
        # define subplot
        plt.subplot(n, n, 1+i)
        plt.axis('off')
        plt.imshow(examples[i, :, :, 0], cmap='gray_r')

    # save plot to file
    filename = f"generated_plot_epoch{epoch+1}.png"
    plt.savefig(filename)
    plt.close()

In [ ]:
def summarise_performance(epoch,
                          g_model:Model,
                          d_model:Model,
                          dataset:np.ndarray,
                          latent_dim,
                          n_samples=150):
    # get real samples
    xreal, yreal = generate_real_samples(dataset, n_samples)

    # evaluate discriminator on real samples
    loss_real, acc_real = d_model.evaluate(xreal, yreal, verbose=0)

    # prepare fake samples
    xfake, yfake = generate_fake_samples(g_model, latent_dim, n_samples)

    # eval discrimiantor on fake samples
    loss_fake, acc_fake = d_model.evaluate(xfake, yfake, verbose=0)

    # summarise discrimiantor performance
    print(f"Epoch={epoch+1} Discriminator Accuracy  "
          f"Real={acc_real*100} and "
          f"Fake={acc_fake*100}")

    # save the plot
    save_plot(xfake, epoch)

    # save the generaor model to file
    g_model.save(f"generator_model_epoch{epoch+1}.hdf5")

In [ ]:
# train the generator and the discriminator
def train(g_model:Model,
          d_model:Model,
          gan_model:Model,
          dataset:np.ndarray,
          latent_dim: int,
          n_epochs=200,
          n_batch=128):

    # evaluate batch per epoch
    bat_per_epo = int(dataset.shape[0] / n_batch)

    # calculate half-batch
    half_batch = int(n_batch / 2)

    # manually enumerate epochs
    for i in range(n_epochs):

        # enumerate bacthes over the trainign set
        for j in range(bat_per_epo):
            # get randomly selected real samples
            xreal, yreal = generate_real_samples(dataset, half_batch)

            # get fake examples
            xfake, yfake = generate_fake_samples(g_model, latent_dim, half_batch)

            # update the discrinator on real images
            d_loss1, d_acc1 = d_model.train_on_batch(xreal, yreal)

            # update discriminator model weights
            d_loss2, d_acc2 = d_model.train_on_batch(xfake, yfake)

            # prepare points in the latent space as inputs for the generator
            xgan = generate_latent_points(latent_dim, n_batch)

            # create inverted labels for the fake samples
            ygan = np.ones((n_batch, 1))

            # update the generator via the discriminators error
            g_loss = gan_model.train_on_batch(xgan, ygan)

            # summarise the loss in this batch
            print(f"Train Epoch: {i+1}, {j+1}/{bat_per_epo} d1={d_loss1}, "
                  f"d2={d_loss2} g={g_loss}")

        if i % 10 == 0:
            summarise_performance(i, g_model, d_model, dataset, latent_dim)

In [ ]:
latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()
train(g_model, d_model, gan_model, dataset, latent_dim)